In [1]:
from datasets import load_dataset

dataset = load_dataset("imdb")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

sample = dataset['train'][0]['text']
encoded = tokenizer(sample, padding="max_length", truncation=True, max_length=128)
print(encoded)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

{'input_ids': [101, 1045, 12524, 1045, 2572, 8025, 1011, 3756, 2013, 2026, 2678, 3573, 2138, 1997, 2035, 1996, 6704, 2008, 5129, 2009, 2043, 2009, 2001, 2034, 2207, 1999, 3476, 1012, 1045, 2036, 2657, 2008, 2012, 2034, 2009, 2001, 8243, 2011, 1057, 1012, 1055, 1012, 8205, 2065, 2009, 2412, 2699, 2000, 4607, 2023, 2406, 1010, 3568, 2108, 1037, 5470, 1997, 3152, 2641, 1000, 6801, 1000, 1045, 2428, 2018, 2000, 2156, 2023, 2005, 2870, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1996, 5436, 2003, 8857, 2105, 1037, 2402, 4467, 3689, 3076, 2315, 14229, 2040, 4122, 2000, 4553, 2673, 2016, 2064, 2055, 2166, 1012, 1999, 3327, 2016, 4122, 2000, 3579, 2014, 3086, 2015, 2000, 2437, 2070, 4066, 1997, 4516, 2006, 2054, 1996, 2779, 25430, 14728, 2245, 2055, 3056, 2576, 3314, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Extract text and labels from Hugging Face dataset
train_texts = dataset['train']['text']
train_labels = dataset['train']['label']
test_texts = dataset['test']['text']
test_labels = dataset['test']['label']

# Tokenizer: map words to integers
max_words = 20000   # limit vocab size to top 20k words
max_len = 200       # each review padded/truncated to 200 tokens

tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(train_texts)

# Convert text to sequences of integers
X_train = tok.texts_to_sequences(train_texts)
X_test = tok.texts_to_sequences(test_texts)

# Pad sequences so they’re all the same length
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

# Labels
y_train = train_labels
y_test = test_labels

print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)


Training set shape: (25000, 200)
Test set shape: (25000, 200)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Model definition
model = Sequential([
    Embedding(input_dim=20000, output_dim=128, input_length=200),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(1, activation='sigmoid')
])

# Compile
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [5]:
import numpy as np

y_train = np.array(y_train)
y_test = np.array(y_test)


In [6]:
history = model.fit(
    X_train, y_train,
    epochs=3,
    batch_size=64,
    validation_split=0.2
)


Epoch 1/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 225s 685ms/step - accuracy: 0.6999 - loss: 0.5606 - val_accuracy: 0.6848 - val_loss: 0.7465
Epoch 2/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 257s 688ms/step - accuracy: 0.8725 - loss: 0.3126 - val_accuracy: 0.6602 - val_loss: 0.9267
Epoch 3/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 265s 698ms/step - accuracy: 0.9184 - loss: 0.2260 - val_accuracy: 0.7080 - val_loss: 0.7267


In [10]:
# Evaluate on test set
score = model.evaluate(X_test, y_test, batch_size=64)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

from sklearn.metrics import classification_report, confusion_matrix

y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred, target_names=["Negative", "Positive"]))
print(confusion_matrix(y_test, y_pred))



391/391 ━━━━━━━━━━━━━━━━━━━━ 49s 124ms/step - accuracy: 0.8831 - loss: 0.2985
Test loss: 0.44631659984588623
Test accuracy: 0.8223599791526794
782/782 ━━━━━━━━━━━━━━━━━━━━ 87s 111ms/step
              precision    recall  f1-score   support

    Negative       0.77      0.91      0.84     12500
    Positive       0.89      0.73      0.80     12500

    accuracy                           0.82     25000
   macro avg       0.83      0.82      0.82     25000
weighted avg       0.83      0.82      0.82     25000

[[11397  1103]
 [ 3338  9162]]


In [11]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Load PyTorch BERT model
bert_model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2
)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Convert Hugging Face Arrow column to list
train_texts = list(dataset['train']['text'])
test_texts = list(dataset['test']['text'])

# Now tokenize
train_encodings = tokenizer(
    train_texts,
    truncation=True,
    padding=True,
    max_length=128,
    return_tensors='tf'
)

test_encodings = tokenizer(
    test_texts,
    truncation=True,
    padding=True,
    max_length=128,
    return_tensors='tf'
)

# Labels as tensors
import tensorflow as tf
train_labels = tf.convert_to_tensor(list(dataset['train']['label']))
test_labels = tf.convert_to_tensor(list(dataset['test']['label']))


TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


In [13]:
import tensorflow as tf

# Training dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        "input_ids": train_encodings["input_ids"],
        "attention_mask": train_encodings["attention_mask"]
    },
    train_labels
)).shuffle(10000).batch(16)

# Test dataset
test_dataset = tf.data.Dataset.from_tensor_slices((
    {
        "input_ids": test_encodings["input_ids"],
        "attention_mask": test_encodings["attention_mask"]
    },
    test_labels
)).batch(16)


In [15]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

# -----------------------------
# Re-tokenize with max_length=256
# -----------------------------
train_texts = list(dataset['train']['text'])
test_texts = list(dataset['test']['text'])

train_encodings = tokenizer(
    train_texts,
    truncation=True,
    padding=True,
    max_length=256,
    return_tensors='np'   # numpy arrays so we can convert to PyTorch
)

test_encodings = tokenizer(
    test_texts,
    truncation=True,
    padding=True,
    max_length=256,
    return_tensors='np'
)

# Convert labels and encodings to PyTorch tensors
train_input_ids = torch.tensor(train_encodings['input_ids'])
train_attention_mask = torch.tensor(train_encodings['attention_mask'])
train_labels_pt = torch.tensor(dataset['train']['label'])

test_input_ids = torch.tensor(test_encodings['input_ids'])
test_attention_mask = torch.tensor(test_encodings['attention_mask'])
test_labels_pt = torch.tensor(dataset['test']['label'])

# Dataset + DataLoader
train_dataset = TensorDataset(train_input_ids, train_attention_mask, train_labels_pt)
test_dataset = TensorDataset(test_input_ids, test_attention_mask, test_labels_pt)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

# -----------------------------
# Model setup
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)

optimizer = AdamW(bert_model.parameters(), lr=2e-5)
loss_fn = CrossEntropyLoss()

# -----------------------------
# Training loop (3 epochs)
# -----------------------------
epochs = 3
for epoch in range(epochs):
    bert_model.train()
    loop = tqdm(train_loader, leave=True)
    total_loss, correct, total = 0, 0, 0

    for batch in loop:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        labels = labels.long()  # CrossEntropyLoss needs long dtype

        optimizer.zero_grad()
        outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()

        # stats
        total_loss += loss.item()
        preds = torch.argmax(logits, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=total_loss/len(train_loader), acc=correct/total)

# -----------------------------
# Evaluation on test set
# -----------------------------
bert_model.eval()
correct, total = 0, 0
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        labels = labels.long()

        outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        correct += (preds == labels).sum().item()
        total += labels.size(0)

test_acc = correct / total
print(f"Test Accuracy: {test_acc:.4f}")

from sklearn.metrics import classification_report, confusion_matrix

all_preds = []
all_labels = []

bert_model.eval()
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print(classification_report(all_labels, all_preds, target_names=["Negative", "Positive"]))
print(confusion_matrix(all_labels, all_preds))



Epoch 3: 100%|██████████| 1563/1563 [19:01<00:00,  1.37it/s, acc=0.977, loss=0.0713]


Test Accuracy: 0.9211
              precision    recall  f1-score   support

    Negative       0.93      0.91      0.92     12500
    Positive       0.91      0.93      0.92     12500

    accuracy                           0.92     25000
   macro avg       0.92      0.92      0.92     25000
weighted avg       0.92      0.92      0.92     25000

[[11400  1100]
 [  873 11627]]
